In [1]:
import keras
from keras.layers import AtrousConv1D, Conv1D, LSTM
import numpy as np

Using TensorFlow backend.


In [2]:
import data_loader as dl

In [3]:
data = dl.DataLoader()

In [4]:
X, Y, m = data.load()

In [5]:
from keras.preprocessing.sequence import pad_sequences

In [6]:
X_pad = pad_sequences(X, maxlen=m, padding='post')
Y_pad = pad_sequences(Y, maxlen=m, padding='post')

In [7]:
sample_weights = np.ones((273, m))
for i in xrange(273):
    for j in xrange(m):
        if (X_pad[i][j] == np.zeros(12)).all():
            sample_weights[i][j] = 0

In [8]:
from keras.models import Sequential

In [9]:
model = Sequential()

In [10]:
from keras.layers import Dense
from keras.layers import Masking
from keras.layers.wrappers import TimeDistributed

In [11]:
from keras.layers import Embedding

In [12]:
mask = np.zeros(12)
# model.add(Masking(mask_value=mask, input_shape=(m, 12)))
model.add(Conv1D(100, 5, border_mode='same', input_shape=(m, 12)))
model.add(Conv1D(50, 5, border_mode='same'))
model.add(Conv1D(25, 5, border_mode='same'))
model.add(Conv1D(12, 5, border_mode='same'))
model.add(LSTM(10, return_sequences=True))
model.add(TimeDistributed(Dense(12, activation="softmax")))

In [13]:
from keras.utils.np_utils import to_categorical

In [14]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              sample_weight_mode='temporal')

In [15]:
X_train, X_test = X_pad[:136, :], X_pad[136:, :]

In [16]:
Y_train, Y_test = Y_pad[:136, :], Y_pad[136:, :]

In [17]:
sample_weights_train, sample_weights_test = sample_weights[:136, :], sample_weights[136:, :]

In [18]:
# # for custom metrics
import numpy as K

# def weighted_accuracy(y_true, y_pred):
#     score_array *= sample_weights
# score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))

def weighted_accuracy(y_true, y_pred):
    # Only for testing
#     score_array = K.equal(K.argmax(y_true, axis=-1),
#                           K.argmax(y_pred, axis=-1))
#     score_array *= weights
#     score_array /= K.mean(K.cast(K.not_equal(weights, 0), K.floatx()))
#     return K.mean(score_array)
    total = 0.0
    count = 0.0
    for i, y_i in enumerate(y_true):
        for j, y_ij in enumerate(y_i):
            if sum(y_ij) > 0:
                total += y_ij[y_pred[i][j]]
                count += 1
    return total / count

In [19]:
model.layers[1]

In [20]:
hist = model.fit(X_train, Y_train, batch_size=136, nb_epoch=100, sample_weight=sample_weights_train)

Epoch 1/100
136/136 [==============================] - 21s - loss: 2.4851
Epoch 2/100
136/136 [==============================] - 21s - loss: 2.4760
Epoch 3/100
136/136 [==============================] - 20s - loss: 2.4477
Epoch 4/100
136/136 [==============================] - 20s - loss: 2.4182
Epoch 5/100
136/136 [==============================] - 20s - loss: 2.3884
Epoch 6/100
136/136 [==============================] - 19s - loss: 2.3698
Epoch 7/100
136/136 [==============================] - 20s - loss: 2.3508
Epoch 8/100
136/136 [==============================] - 22s - loss: 2.3313
Epoch 9/100
136/136 [==============================] - 27s - loss: 2.3085
Epoch 10/100
136/136 [==============================] - 21s - loss: 2.2987
Epoch 11/100
136/136 [==============================] - 22s - loss: 2.2906
Epoch 12/100
136/136 [==============================] - 21s - loss: 2.2892
Epoch 13/100
136/136 [==============================] - 21s - loss: 2.2696
Epoch 14/100
136/136 [============

In [21]:
Y_prediction = model.predict_classes(X_test, batch_size=5)

137/137 [==============================] - 52s    


In [22]:
# Y_prediction = to_categorical(Y_prediction)
# weighted_accuracy(Y_test, Y_prediction, sample_weights_test)
weighted_accuracy(Y_test, Y_prediction)

0.40619406261609647

In [23]:
model.evaluate(X_test, Y_test, batch_size=137, sample_weight=sample_weights_test)

137/137 [==============================] - 5s


1.9258359670639038

In [24]:
Y_prediction[0]

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
Y_prediction.shape

(137, 4954)

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
len(model.get_weights())

22

In [28]:
count = 0
for y in Y_prediction:
    if sum(y) > 0:
        count += 1
        print sum(y)
print count

300
433
92
545
585
114
3962
1683
169
317
192
275
51
2
84
140
29
79
55
40
685
20
85
90
76
366
92
335
50
60
52
28
258
84
32
52
346
631
290
73
540
761
659
20
46
266
106
218
260
35
47
12
2358
155
182
68
57
50
377
20
70
50
384
132
32
94
24
340
117
8
29
42
40
18
8
118
12
450
138
44
70
557
76
26
16
205
203
115
608
65
58
38
4
14
8
19
87
93
20
45
911
388
235
24
561
137
58
250
46
97
40
52
112
